# Guide utilisation du stockage Onyxia hackathon IA & Mobilités 2024

**<!> Ce guide est destiné aux participants du hackathon et est valable pour les services instanciés dans le groupe "dlb-hackathon" <!>**

**<!> Les paramètres de connexion au stockage sont éphémères, la logique est de supprimer les services en fin de journée, __après sauvegarde de vos fichiers de travail__ <!>**

## Installation du client Minio S3 (stockage distribué)

In [1]:
!pip install s3fs

## Listing des fichiers avec le client S3

In [2]:
import os
import s3fs

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

In [8]:
{key: os.environ[key] for key in os.environ if "AWS" in key}

{'AWS_DEFAULT_REGION': 'fr-central',
 'AWS_SESSION_TOKEN': 'XXXXXXX.XXXXXXXXX.XXXXXX-XXXXX',
 'AWS_S3_ENDPOINT': 'minio.data-platform-self-service.net',
 'AWS_ACCESS_KEY_ID': 'xxx',
 'AWS_SECRET_ACCESS_KEY': 'xxx+xxxxx++xxxx'}

In [ ]:
fs.ls("dlb-hackathon/datasets-diffusion/reclamations_formulaire_web", refresh=True)

['dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/.keep',
 'dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet']

## Dépôt de fichier sur un répertoire d'équipe avec la ligne de commande

In [51]:
# On peut uploader par une commande terminal
!mc cp ./export_overture.parquet s3/dlb-hackathon/equipe-0/export_overture.parquet

...re.parquet: 240.75 KiB / 240.75 KiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 4.61 MiB/s 0s

## Requêtage de données avec Duckdb

### Paramétrage de duckdb depuis les variables d'environnement

In [11]:
%pip install duckdb

Note: you may need to restart the kernel to use updated packages.


In [14]:
# Paramétrage pour un appel Duckdb
import duckdb as ddb
ddb.execute("SET s3_region='fr-central';")
ddb.execute("SET s3_url_style='path';")
ddb.execute("SET s3_endpoint='minio.data-platform-self-service.net';")
ddb.execute(f"SET s3_access_key_id='{os.environ["AWS_ACCESS_KEY_ID"]}' ;") # Aussi récupérable dans les paramètres "Valeurs de Helm" du service
ddb.execute(f"SET s3_secret_access_key='{os.environ["AWS_SECRET_ACCESS_KEY"]}';") # Aussi récupérable dans les paramètres "Valeurs de Helm" du service

### Requêtage d'un fichier parquet

In [16]:
ddb.sql("""
select * from "s3://dlb-hackathon/datasets-diffusion/reclamations_formulaire_web/idfm_reclamations_formulaire_web.parquet" 
""").to_df().sample(5)

,id,date_demande,canal_demande,texte_anonymise,classification_services_idfm
11803,46342,2024-07-08 12:28:00,Formulaire Web,"Bonjour,j'ai pris des tickets tarif réduit pou...",Tarification Abonnement > Support Canal Mobile...
10358,27879,2023-03-18 15:47:00,Formulaire Web,Bonjour. \n J'espère que vous allez très bien....,Tarification Abonnement > Campagnes de dédomma...
3798,36955,2023-10-09 09:26:00,Formulaire Web,"Bonjour , j'ai acheté une carte en juin dernie...",Tarification Abonnement > Support Autre > Pass...
9126,43086,2024-03-29 18:12:00,Formulaire Web,"Bonjour, je reçois les attestations de paiemen...",Tarification Abonnement > Support Autre > Pass...
17568,27259,2023-03-13 11:25:00,Formulaire Web,"Bonjour, \n \n J'ai fait une demande de navig...",Tarification Abonnement > Liberte + > Données ...


## Manipulation des données avec Pandas

In [17]:
!pip install pandas

### Lecture d'un fichier

In [18]:
import pandas as pd

BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "datasets-diffusion/ascenseurs_historique_etat/RELEVES_ETATS_ASCENSEURS_SNCF_RATP_2021-2024.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in)

/tmp/ipykernel_4015/2968963640.py:8: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_in)


In [19]:
df.head()

,operateur,date_releve,code_appareil,code_fonctionnel,etat,date_etat_debut,date_etat_fin,emplacement_gare,emplacement_situation,emplacement_direction
0,RATP,2021-01-01 08:07:57,1004.0,ASC01-EQ11-A005-000000269,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
1,RATP,2021-01-01 08:07:57,1005.0,ASC02-EQ11-A005-000000270,2.0,01/01/2021,02/01/2021,SAINT OUEN,SA200,NaN
2,RATP,2021-01-01 08:07:57,1006.0,ASC03-EQ11-A005-000000271,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
3,RATP,2021-01-01 08:07:57,1007.0,ASC04-EQ11-A005-000000272,2.0,01/01/2021,NaN,SAINT OUEN,SA200,NaN
4,RATP,2021-01-01 08:07:57,1008.0,ASC05-EQ11-A005-000000273,2.0,01/01/2021,02/01/2021,SAINT OUEN,SA200,NaN


### Ecriture d'un fichier

In [20]:
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "datasets-diffusion/donnees_territoire_GPSEO/bornes_recharges/IRVE_CU.parquet"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_parquet(file_in)

In [21]:
# Ecrire
BUCKET_OUT = "dlb-hackathon"
FILE_KEY_OUT_S3 = "equipe-0/donnees_bornes_recharges.csv"
FILE_PATH_OUT_S3 = BUCKET_OUT + "/" + FILE_KEY_OUT_S3

with fs.open(FILE_PATH_OUT_S3, 'w') as file_out:
    df.to_csv(file_out)

In [22]:
# Verification
BUCKET = "dlb-hackathon"
FILE_KEY_S3 = "equipe-0/donnees_bornes_recharges.csv"
FILE_PATH_S3 = BUCKET + "/" + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    df = pd.read_csv(file_in)

df.head()

,Unnamed: 0,objectid,adresse_ir,ville,nom_statio,geom
0,0,4,Avenue De La Division Leclerc,AUBERGENVILLE,AUBERGENVILLE - Avenue De La Division Leclerc,"b""\x01\x01\x00\x00\x00L\x83\xc0\x16\x8e\xb6\xf..."
1,1,5,Place François Mitterrand,AUBERGENVILLE,AUBERGENVILLE - Place François Mitterrand,b'\x01\x01\x00\x00\x00P\xe6\xbb\xa2\xf8\x93\xf...
2,2,6,Place De La Mairie,AULNAY-SUR-MAULDRE,AULNAY-SUR-MAULDRE - Place De La Mairie,b'\x01\x01\x00\x00\x00m\x17\xd1$\xa0\x80\xfd?\...
3,3,7,2 Rue De Charnelles,BOUAFLE,BOUAFLE - Rue De Charnelles,b'\x01\x01\x00\x00\x00\xc3@\xe9\xd5\x93T\xfe?\...
4,4,8,Rue Du Béarn,BUCHELAY,BUCHELAY - Rue Du Béarn,"b""\x01\x01\x00\x00\x00'\xf2PN,\xab\xfa?k,j\xb3..."


## Listing des données spécifiques au Hackathon

In [26]:
import pandas as pd

In [ ]:
root_path = "dlb-hackathon/datasets-diffusion/"
catalog = pd.DataFrame(fs.ls("dlb-hackathon/datasets-diffusion/", refresh=True), columns=['file_path'])
ddb.sql(f"""create or replace table tmp_paths as select '{root_path}' parent, file_path path from catalog""")

for path in catalog["file_path"] :
    catalog = pd.DataFrame(fs.ls(path, refresh=True), columns=['file_path'])
    ddb.sql(f"""insert into tmp_paths select '{path}' parent, file_path path from catalog""")


In [49]:
ddb.sql("copy(select * from tmp_paths) to catalogue_hackathon.csv")

In [40]:
catalog

,file_path
0,dlb-hackathon/datasets-diffusion/.keep
1,dlb-hackathon/datasets-diffusion/ascenseurs_hi...
2,dlb-hackathon/datasets-diffusion/courses_trans...
3,dlb-hackathon/datasets-diffusion/documents_idf...
4,dlb-hackathon/datasets-diffusion/documents_nor...
5,dlb-hackathon/datasets-diffusion/documents_nor...
6,dlb-hackathon/datasets-diffusion/donnees_terri...
7,dlb-hackathon/datasets-diffusion/equipements_g...
8,dlb-hackathon/datasets-diffusion/historique_in...
9,dlb-hackathon/datasets-diffusion/recherches_it...
